# Weather Classification and SabIA's Friendly Forecasts

This notebook demonstrates the process of classifying weather conditions based on forecast data and generating friendly, culturally-contextualized weather messages using the SabIA virtual assistant (powered by OpenAI's ChatGPT).

## Table of Contents

1.  [Setup and Data Loading](#setup-and-data-loading)
2.  [Weather Classification](#weather-classification)
3.  [SabIA Message Generation and Saving Results](#sabia-message-generation)

---

## 1. Setup and Data Loading

This section handles the initial setup, including installing necessary libraries and loading the weather forecast data. The `WeatherClassifier` class reads various CSV files containing weather predictions (temperature, humidity, wind speed) from the `forecast/` directory.


In [1]:
!pip install openai

## 2. Weather Classification

This section uses the `WeatherClassifier` to process the loaded forecast data and classify each day according to predefined criteria. The classification results include categories like "very hot," "very cold," "very windy," "very wet," "very uncomfortable," and a default "normal" if no extreme conditions are met. Additionally, city, latitude, longitude, and all relevant prediction values are included.


In [2]:
from weather_classifier import WeatherClassifier
import os

# Instanciar o classificador (com valores padrão para Uberlândia)
classifier = WeatherClassifier(city_name='Uberlândia', latitude=-18.9186, longitude=-48.2772)

# Classificar o tempo
classified_results = classifier.classify_weather()

# Exibir os resultados
print("Resultados da Classificação do Tempo:")
print(classified_results)

# Salvar os resultados em um arquivo CSV na nova pasta
output_folder = "classified_forecasts"
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "classified_weather_forecast.csv")
classified_results.to_csv(output_path, index=False)
print(f"\nResultados salvos em: {output_path}")


Resultados da Classificação do Tempo:
           date classification   city_name  latitude  longitude  \
0    2025-10-02         normal  Uberlândia  -18.9186   -48.2772   
1    2025-10-03       very_hot  Uberlândia  -18.9186   -48.2772   
2    2025-10-04       very_hot  Uberlândia  -18.9186   -48.2772   
3    2025-10-05       very_hot  Uberlândia  -18.9186   -48.2772   
4    2025-10-06       very_hot  Uberlândia  -18.9186   -48.2772   
..          ...            ...         ...       ...        ...   
176  2026-03-27         normal  Uberlândia  -18.9186   -48.2772   
177  2026-03-28         normal  Uberlândia  -18.9186   -48.2772   
178  2026-03-29         normal  Uberlândia  -18.9186   -48.2772   
179  2026-03-30         normal  Uberlândia  -18.9186   -48.2772   
180  2026-03-31         normal  Uberlândia  -18.9186   -48.2772   

     T2M_prediction  T2M_MAX_prediction  T2M_MIN_prediction  WS2M_prediction  \
0               NaN                 NaN                 NaN              NaN 

### Classification Logic Details

The `WeatherClassifier` class uses the following rules to categorize weather conditions for each day's forecast:

*   **Very Hot (is_very_hot)**:
    *   Condition: Maximum predicted temperature (`T2M_MAX_prediction`) is greater than 30°C.
*   **Very Cold (is_very_cold)**:
    *   Condition: Minimum predicted temperature (`T2M_MIN_prediction`) is less than 10°C.
*   **Very Windy (is_very_windy)**:
    *   Condition: Predicted wind speed (`WS2M_prediction`) is greater than 10 m/s.
*   **Very Wet (is_very_wet)**:
    *   Condition: Predicted relative humidity (`RH2M_prediction`) is greater than 90%.
*   **Very Uncomfortable (is_very_uncomfortable)**:
    *   Condition: Predicted average temperature (`T2M_prediction`) is greater than 28°C **AND** predicted relative humidity (`RH2M_prediction`) is greater than 80%.
*   **Normal**:
    *   Condition: None of the above extreme conditions are met. This serves as a default classification to ensure every day has at least one category.

For days with multiple extreme conditions, all applicable classifications will be listed (e.g., "very_hot, very_uncomfortable"). The SabIA message generation then prioritizes these extreme classifications to create a relevant and personalized message.


In [3]:
from weather_classifier import WeatherClassifier

# Instanciar o classificador
classifier = WeatherClassifier()

# Classificar o tempo
classified_results = classifier.classify_weather()

# Exibir os resultados
print("Resultados da Classificação do Tempo:")
print(classified_results)


Resultados da Classificação do Tempo:
           date classification   city_name  latitude  longitude  \
0    2025-10-02         normal  Uberlândia  -18.9186   -48.2772   
1    2025-10-03       very_hot  Uberlândia  -18.9186   -48.2772   
2    2025-10-04       very_hot  Uberlândia  -18.9186   -48.2772   
3    2025-10-05       very_hot  Uberlândia  -18.9186   -48.2772   
4    2025-10-06       very_hot  Uberlândia  -18.9186   -48.2772   
..          ...            ...         ...       ...        ...   
176  2026-03-27         normal  Uberlândia  -18.9186   -48.2772   
177  2026-03-28         normal  Uberlândia  -18.9186   -48.2772   
178  2026-03-29         normal  Uberlândia  -18.9186   -48.2772   
179  2026-03-30         normal  Uberlândia  -18.9186   -48.2772   
180  2026-03-31         normal  Uberlândia  -18.9186   -48.2772   

     T2M_prediction  T2M_MAX_prediction  T2M_MIN_prediction  WS2M_prediction  \
0               NaN                 NaN                 NaN              NaN 

## 3. SabIA Message Generation and Saving Results

This section integrates with OpenAI's ChatGPT (using the `gpt-4o-mini` model) to generate engaging, culturally-rich weather messages from the perspective of SabIA, a friendly weather assistant from Uberlândia, Minas Gerais. The messages are personalized for each day's classification and include local charm and Cerrado nature references. Finally, all results, including the generated messages, are saved to a CSV file.


In [4]:
classified_results['classification'].value_counts()

classification
normal      127
very_hot     54
Name: count, dtype: int64

In [5]:
classified_results.tail()

,date,classification,city_name,latitude,longitude,T2M_prediction,T2M_MAX_prediction,T2M_MIN_prediction,WS2M_prediction,RH2M_prediction
176,2026-03-27,normal,Uberlândia,-18.9186,-48.2772,25.272703,28.636410,17.669810,3.664848,40.351490
177,2026-03-28,normal,Uberlândia,-18.9186,-48.2772,25.313677,28.507050,17.935743,2.938153,40.401920
178,2026-03-29,normal,Uberlândia,-18.9186,-48.2772,25.252794,28.483498,18.147354,3.707678,39.374805
179,2026-03-30,normal,Uberlândia,-18.9186,-48.2772,25.399710,28.596664,18.145632,2.640987,38.894234
180,2026-03-31,normal,Uberlândia,-18.9186,-48.2772,25.342714,28.569030,17.868454,3.312911,39.330883


In [6]:
classified_results

,date,classification,city_name,latitude,longitude,T2M_prediction,T2M_MAX_prediction,T2M_MIN_prediction,WS2M_prediction,RH2M_prediction
0,2025-10-02,normal,Uberlândia,-18.9186,-48.2772,NaN,NaN,NaN,NaN,NaN
1,2025-10-03,very_hot,Uberlândia,-18.9186,-48.2772,26.586975,34.238174,19.585539,2.350225,32.971558
2,2025-10-04,very_hot,Uberlândia,-18.9186,-48.2772,26.465452,33.050660,19.880203,2.880506,31.746767
3,2025-10-05,very_hot,Uberlândia,-18.9186,-48.2772,26.492819,32.951004,20.508945,2.671745,31.476688
4,2025-10-06,very_hot,Uberlândia,-18.9186,-48.2772,26.642122,33.282837,21.604950,2.869747,32.120575
...,...,...,...,...,...,...,...,...,...,...
176,2026-03-27,normal,Uberlândia,-18.9186,-48.2772,25.272703,28.636410,17.669810,3.664848,40.351490
177,2026-03-28,normal,Uberlândia,-18.9186,-48.2772,25.313677,28.507050,17.935743,2.938153,40.401920
178,2026-03-29,normal,Uberlândia,-18.9186,-48.2772,25.252794,28.483498,18.147354,3.707678,39.374805
179,2026-03-30,normal,Uberlândia,-18.9186,-48.2772,25.399710,28.596664,18.145632,2.640987,38.894234


In [7]:
import openai

# --- CONFIGURAÇÃO DO CHATGPT (VOCÊ PRECISARÁ ATUALIZAR ISSO) ---
# Insira sua chave de API real do OpenAI aqui, substituindo o placeholder.
openai.api_key = "API-KEY" # <-- COLOQUE SUA CHAVE AQUI!

def get_chatgpt_sabia_message(row):
    date = row['date']
    classification_str = row['classification']
    t2m_pred = row['T2M_prediction']
    t2m_max_pred = row['T2M_MAX_prediction']
    t2m_min_pred = row['T2M_MIN_prediction']
    ws2m_pred = row['WS2M_prediction']
    rh2m_pred = row['RH2M_prediction']
    city = row['city_name']

    # Priorizar a primeira classificação extrema, ou usar 'normal'
    classifications = [c.strip() for c in classification_str.split(',')] if classification_str else ['normal']
    main_classification = 'normal'
    for c in ['very_hot', 'very_cold', 'very_windy', 'very_wet', 'very_uncomfortable']:
        if c in classifications:
            main_classification = c
            break

    # Dicionário de prompts específicos para cada classe de clima
    specific_prompts = {
        'very_hot': f"It's a scorching day in Uberlândia! The perfect weather to enjoy a refreshing dip at Parque do Sabiá's pools or cool off with some delicious açaí. Stay hydrated!",
        'very_cold': f"Brr! A very cold day in Uberlândia. How about a warm pão de queijo and a cozy afternoon enjoying the tranquility of Praça Tubal Vilela from a nice cafe?",
        'very_windy': f"Hold onto your hats, it's windy in Uberlândia! A great day for a walk in the open areas of Parque do Sabiá, just watch out for your hair!",
        'very_wet': f"Rainy day in Uberlândia! Perfect for visiting the Municipal Museum or catching a show at the Oscar Niemeyer-designed Teatro Municipal. Don't forget your umbrella!",
        'very_uncomfortable': f"A truly uncomfortable day in Uberlândia due to high heat and humidity. Best to stay indoors, enjoy some traditional Minas Gerais coffee, and maybe plan your next visit to the Mercado Municipal!",
        'normal': f"A pleasant day in Uberlândia! Perfect for an outdoor stroll at Parque do Sabiá or exploring the vibrant local markets. Enjoy the mild Cerrado weather!"
    }

    # Selecionar o prompt base
    base_message = specific_prompts.get(main_classification, specific_prompts['normal'])

    # Construir o prompt final para o ChatGPT
    prompt = f"You are SabIA, a friendly and concise weather assistant from Uberlândia, Minas Gerais, Brazil. Uberlândia is famous for its Parque do Sabiá, Praça Tubal Vilela, and the Oscar Niemeyer-designed Teatro Municipal, and is located in the unique Cerrado biome. Your persona is inspired by the warm, welcoming, and unique spirit of Minas Gerais. For {city} on {date}, the forecast is: average temperature {t2m_pred:.1f}°C (min: {t2m_min_pred:.1f}°C, max: {t2m_max_pred:.1f}°C), wind at {ws2m_pred:.1f} m/s, and humidity at {rh2m_pred:.1f}%. " \
             f"The general conditions are: {classification_str}. " \
             f"Based on this, and reflecting Uberlândia's charm, Cerrado nature, and the local Minas Gerais culture, create a short, friendly message (maximum 150 characters) and a suggested activity. Your response should be ONLY in English."

    # --- CHAMADA REAL À API DO CHATGPT ---
    client = openai.OpenAI(api_key=openai.api_key)
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # Usando o modelo gpt-4o-mini
            messages=[{"role": "user", "content": prompt}],
            max_tokens=75,  # Ajustado para acomodar a mensagem mais rica
            temperature=0.8 # Aumentado para mais criatividade
        )
        chatgpt_message = response.choices[0].message.content.strip()
        
    except Exception as e:
        chatgpt_message = f"SabIA: Oops! Could not connect for forecast. Error: {e}"

    # Garantir que a mensagem final seja curta (max 150 caracteres)
    if len(chatgpt_message) > 150:
        chatgpt_message = chatgpt_message[:147] + "..."

    return chatgpt_message

print("\nMensagens do SabIA, seu assistente de clima (via ChatGPT):")

# Lista para armazenar as mensagens geradas
sabia_messages = []

# Itera sobre TODAS as linhas para gerar as mensagens
for index, row in classified_results.iterrows():
    en_msg = get_chatgpt_sabia_message(row)
    sabia_messages.append(en_msg)
    # Opcional: imprimir cada mensagem enquanto é gerada (pode ser lento para muitos dados)
    # print(f"Data: {row['date']} - EN: {en_msg}")

# Adicionar as mensagens como uma nova coluna ao DataFrame
classified_results['sabia_message_en'] = sabia_messages

# Salvar o DataFrame atualizado em um novo arquivo CSV
output_folder = "classified_forecasts"
output_path_with_messages = os.path.join(output_folder, "classified_weather_forecast_with_messages.csv")
classified_results.to_csv(output_path_with_messages, index=False)
print(f"\nResultados classificados com mensagens do SabIA salvos em: {output_path_with_messages}")

# Opcional: Exibir as primeiras linhas do DataFrame com as mensagens
print("\nPrimeiras linhas do DataFrame com mensagens do SabIA:")
print(classified_results.head())



Mensagens do SabIA, seu assistente de clima (via ChatGPT):

Resultados classificados com mensagens do SabIA salvos em: classified_forecasts\classified_weather_forecast_with_messages.csv

Primeiras linhas do DataFrame com mensagens do SabIA:
         date classification   city_name  latitude  longitude  T2M_prediction  \
0  2025-10-02         normal  Uberlândia  -18.9186   -48.2772             NaN   
1  2025-10-03       very_hot  Uberlândia  -18.9186   -48.2772       26.586975   
2  2025-10-04       very_hot  Uberlândia  -18.9186   -48.2772       26.465452   
3  2025-10-05       very_hot  Uberlândia  -18.9186   -48.2772       26.492819   
4  2025-10-06       very_hot  Uberlândia  -18.9186   -48.2772       26.642122   

   T2M_MAX_prediction  T2M_MIN_prediction  WS2M_prediction  RH2M_prediction  \
0                 NaN                 NaN              NaN              NaN   
1           34.238174           19.585539         2.350225        32.971558   
2           33.050660           19

In [8]:
classified_results

,date,classification,city_name,latitude,longitude,T2M_prediction,T2M_MAX_prediction,T2M_MIN_prediction,WS2M_prediction,RH2M_prediction,sabia_message_en
0,2025-10-02,normal,Uberlândia,-18.9186,-48.2772,NaN,NaN,NaN,NaN,NaN,Enjoy a lovely day in Uberlândia! Explore Parq...
1,2025-10-03,very_hot,Uberlândia,-18.9186,-48.2772,26.586975,34.238174,19.585539,2.350225,32.971558,Hello! Expect a very hot day at 26.6°C in Uber...
2,2025-10-04,very_hot,Uberlândia,-18.9186,-48.2772,26.465452,33.050660,19.880203,2.880506,31.746767,"Hello, sunny Uberlândia! With temps reaching 3..."
3,2025-10-05,very_hot,Uberlândia,-18.9186,-48.2772,26.492819,32.951004,20.508945,2.671745,31.476688,"Hello, friend! It's a very hot day in Uberlând..."
4,2025-10-06,very_hot,Uberlândia,-18.9186,-48.2772,26.642122,33.282837,21.604950,2.869747,32.120575,Hello! It's a very hot day in Uberlândia at 26...
...,...,...,...,...,...,...,...,...,...,...,...
176,2026-03-27,normal,Uberlândia,-18.9186,-48.2772,25.272703,28.636410,17.669810,3.664848,40.351490,Hello! Expect a lovely day in Uberlândia with ...
177,2026-03-28,normal,Uberlândia,-18.9186,-48.2772,25.313677,28.507050,17.935743,2.938153,40.401920,"Good morning, Uberlândia! Enjoy a lovely day a..."
178,2026-03-29,normal,Uberlândia,-18.9186,-48.2772,25.252794,28.483498,18.147354,3.707678,39.374805,Hello! Enjoy a lovely day in Uberlândia with a...
179,2026-03-30,normal,Uberlândia,-18.9186,-48.2772,25.399710,28.596664,18.145632,2.640987,38.894234,Hello! Enjoy a lovely day in Uberlândia with a...
